In [6]:
from dotenv import load_dotenv

# Loading the env variables
load_dotenv(dotenv_path=".env")

True

In [1]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
print(embeddings)

model_name='sentence-transformers/all-mpnet-base-v2' cache_folder=None model_kwargs={} encode_kwargs={} query_encode_kwargs={} multi_process=False show_progress=False


In [9]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="openai/gpt-oss-120b",
    temperature=0,
    max_tokens=None,
    reasoning_format="parsed",
    timeout=None,
    max_retries=2,
    # other params...
)

# Test Invoke
messages = [
    (
        "system",
        "You are a mathematician. Help the users with their queries.",
    ),
    ("human", "Please tell me about taylor's series."),
]
ai_msg = llm.invoke(messages)
print(ai_msg.pretty_repr(html=True))

================================== Ai Message ==================================

## Taylor’s Series – An Overview

A **Taylor series** is a way of representing a smooth (infinitely differentiable) function as an infinite sum of polynomial terms built from the function’s derivatives at a single point.  
It generalizes the idea of a **Maclaurin series**, which is the special case where the expansion point is \(0\).

---

### 1. Formal Definition  

Let \(f\) be a function that has derivatives of all orders at a point \(a\).  
The **Taylor series of \(f\) about \(a\)** is  

\[
\boxed{%
f(x)=\sum_{n=0}^{\infty}\frac{f^{(n)}(a)}{n!}\,(x-a)^{\,n}
}
\qquad\text{(if the series converges to }f(x)\text{).}
\]

- \(f^{(n)}(a)\) denotes the \(n\)-th derivative of \(f\) evaluated at \(a\).  
- \(n!\) is the factorial \(1\cdot2\cdots n\).  
- The term \(\frac{f^{(n)}(a)}{n!}(x-a)^n\) is the **\(n\)-th Taylor polynomial** term.

If we set \(a=0\) we obtain the **Maclaurin series**:

\[
f(x)=\sum_{n

In [13]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import RecursiveUrlLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate

#### INDEXING ####

# Load Documents
loader = RecursiveUrlLoader(
    url="https://dtu.ac.in/Web/About/history.php",
    max_depth=1,  # how deep to follow links
    extractor=lambda x: bs4.BeautifulSoup(x, "html.parser").text,  # clean text
)
docs = loader.load()

# Cleaning the output
for doc in docs:
    doc.page_content = doc.page_content.replace("\n","").replace("\t","")
print(docs)

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# Embed
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=embeddings)

retriever = vectorstore.as_retriever()

[Document(metadata={'source': 'https://dtu.ac.in/Web/About/history.php', 'content_type': 'text/html; charset=UTF-8', 'title': '\n History \xa0| Delhi Technological University', 'language': None}, page_content=' History \xa0| Delhi Technological University  Vision & MissionEast Delhi Campus Wall of DonorsAlumni AffairsInternational AffairsR&D NCEET ICCNIRFPhoto GalleryStudioRTI InfoEngg CellContact Us      About UsHistoryVision & MissionCore ValuesResearchQuality Policy Quality Manual Strategic PlanVisit UsCampus MapIPRContact Us AdministrationChancellorVice ChancellorPro Vice ChancellorCourt Board Of Management Academic Council Planning Board Finance CommitteeDeans Academic HeadAdministrative OfficersOmbudsmanAcademicsAcademic - UGAcademic - PGPrograms offeredScheme & SyllabusAdmission B.Tech.  B.Tech (NRI / FN / PIO)  B.Tech (Lateral Entry)  B.Tech Continuing Education B.Des (Bachelor of Design) BA(H) Economics  BBA   MA Economics  MBA  EMBA M.Tech.  M.Des (Master of Design) M.Sc.   I

C:\Users\karti\AppData\Local\Temp\ipykernel_10404\4202432096.py:37: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents("Tell about DTU")


[Document(metadata={'source': 'https://dtu.ac.in/Web/About/history.php', 'title': '\n History \xa0| Delhi Technological University', 'content_type': 'text/html; charset=UTF-8'}, page_content='almost 55 years and has shifted in 1996 to its lush green sprawling campus of 164 Acres at Bawana Road, adjoining Sector-17, Rohini, Delhi-42. Its shifting to new campus has added the dimension of research and caused innovations in plenty, which has received high national and international acclaim. As Delhi Technological University, it has the desired autonomy to excel and shape itself as a world class Technological University.Upholding and uplifting its long and remarkable history in education and research, the University currently offers various inter-disciplinary and industry relevant programs in Science, Engineering, Management & Allied areas at both the undergraduate and postgraduate level. The university currently offers 21 undergraduate, 36 postgraduate and Ph.D. programs. The ranking of DT

In [14]:
# Test retrieve of document
retrieved_docs = retriever.invoke("Tell about DTU")
print(retrieved_docs)

[Document(metadata={'source': 'https://dtu.ac.in/Web/About/history.php', 'content_type': 'text/html; charset=UTF-8', 'title': '\n History \xa0| Delhi Technological University'}, page_content='almost 55 years and has shifted in 1996 to its lush green sprawling campus of 164 Acres at Bawana Road, adjoining Sector-17, Rohini, Delhi-42. Its shifting to new campus has added the dimension of research and caused innovations in plenty, which has received high national and international acclaim. As Delhi Technological University, it has the desired autonomy to excel and shape itself as a world class Technological University.Upholding and uplifting its long and remarkable history in education and research, the University currently offers various inter-disciplinary and industry relevant programs in Science, Engineering, Management & Allied areas at both the undergraduate and postgraduate level. The university currently offers 21 undergraduate, 36 postgraduate and Ph.D. programs. The ranking of DT

In [ ]:
# Creating the Prompt manaually
template = """Answer all the questions from the following prompts:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [16]:
chain = prompt | llm
question = "Tell me about DTU's history ?"
retrieved_docs = retriever.invoke(question)
chain.invoke({"context":retrieved_docs,"question":question})

AIMessage(content='**Delhi Technological University (DTU) – A Brief Historical Overview**\n\nDelhi Technological University, originally established as the Delhi College of Engineering in 1941, has evolved over the decades into one of India’s premier technical institutions. While the excerpts you provided focus mainly on the university’s recent achievements, they also reflect the broader trajectory of DTU’s growth and its commitment to excellence.\n\n| Milestone | Highlights |\n|-----------|------------|\n| **Founding & Early Years** | Started as Delhi College of Engineering (DCE) in 1941, serving as a hub for engineering education in the capital. |\n| **Transition to University Status** | In 2009, the institution was upgraded to a full‑fledged university and renamed **Delhi Technological University** to reflect its expanded academic and research mandate. |\n| **Academic Expansion** | Today DTU offers undergraduate, postgraduate, and doctoral programmes across engineering, technology, m

In [18]:
# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Use Built-in chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
rag_chain.invoke(question)

'**Delhi Technological University (DTU) – A Brief History**\n\n| Year | Milestone |\n|------|-----------|\n| **1941** | Established as **Delhi College of Engineering (DCE)** under the University of Delhi to meet the demand for technical education in the capital. |\n| **1952** | Shifted to its present 42‑acre campus at **Bawana Road, Delhi**. The campus was designed to accommodate expanding academic and research facilities. |\n| **1960s‑1970s** | Introduced new undergraduate programmes in **Civil, Mechanical, Electrical, and Electronics Engineering** and began offering postgraduate (M.Tech) courses. |\n| **1990** | Became a **deemed‑to‑be university** under the name **Delhi College of Engineering (University)**, gaining greater academic autonomy. |\n| **2000** | Launched the **Institute of Innovation and Incubation (I²I)**, fostering entrepreneurship and start‑up creation among students and faculty. |\n| **2009** | Renamed **Delhi Technological University (DTU)** by an act of the Parlia